# Working with Video Data in Python

This notebook was created for the [YouTube tutorial that you can find here](https://youtu.be/AxIc-vGaHQ0)!!


- What you will learn:
    - What is video data?
    - Converting between video formats
    - Pulling metadata from video file.
    - How to open a video file and extract images using cv2 (openCV).
    - Adding annotations to video frames.
    - Saving edited video.

# What is video data?
Simply put, video data is a series of images that when shown in sucession give the illusion of motion. Like a flipbook!

<img src="https://i.imgur.com/Rx9R3Hf.jpg" width="500" height="500" />




## Video resolution
Video resolution is the size of each image in the video. It doesn't need to be a standard size, but there are common sizes for video.

<img src="https://i.imgur.com/CUo7rGb.png" width="500" height="500" />

## What is frame rate?

Frame rate determines the number of "images" seen per second. Can be described as fps (frames per second) or Hz (general unit for frequency)

<img src="https://i.imgur.com/uSOX932.jpg" width="500" height="500" />

# Getting Setup for working with Video in Python
Packages you'll need installed:
- opencv / cv2 : `pip install opencv-python`
- matplotlib : `pip install matplotlib`
- ffmpeg (for converting videos)

In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

from glob import glob

import IPython.display as ipd
from tqdm.notebook import tqdm

import subprocess

# Converting video types
Use ffmpeg to convert mov to mp4. Read more in the documentation here: https://ffmpeg.org/ffmpeg.html

In [2]:
input_file = 'bdd100k_videos_train_00/bdd100k/videos/train/026c7465-309f6d33.mov'
subprocess.run(['ffmpeg', # Run ffmpeg
                '-i', input_file, # input file
                '-qscale', '0',#quality scale
                '026c7465-309f6d33.mp4']) 

CompletedProcess(args=['ffmpeg', '-i', 'bdd100k_videos_train_00/bdd100k/videos/train/026c7465-309f6d33.mov', '-qscale', '0', '026c7465-309f6d33.mp4'], returncode=0)

# Display Video in Notebook

In [3]:
ipd.Video('026c7465-309f6d33.mp4', width=500)

# Open the Video and Read Metadata

We might want to know the frames, fpt, size of video, etc. This is important info we need to help transform later.

In [4]:
# Load in the video capture
capture = cv2.VideoCapture('026c7465-309f6d33.mp4')

In [9]:
# Total number of frames in video
print(f"We have {capture.get(cv2.CAP_PROP_FRAME_COUNT)} total frames.")

We have 2398.0 total frames.


In [10]:
# Get height and width
height = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = capture.get(cv2.CAP_PROP_FRAME_WIDTH)

print(f"Height: {height}. Width: {width}")

Height: 720.0. Width: 1280.0


In [12]:
# Get framerate of video!
fps = capture.get(cv2.CAP_PROP_FPS)

print(f'Frames per second: {fps:.2f}')

Frames per second: 59.94


We'll want to release the video after it's done being used by python. Do this for all videos.

In [13]:
capture.release()

# Pulling in Images from Video

Remember, video is just a big sequence of images. 

In [ ]:
cap = cv2.VideoCapture('026c7465-309f6d33.mp4')
ret, img = cap.read()
print(f'Returned {ret} and img of shape {img.shape}')

In [ ]:
## Helper function for plotting opencv images in notebook
def display_cv2_img(img, figsize=(10, 10)):
    img_ = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    fig, ax = plt.subplots(figsize=figsize)
    ax.imshow(img_)
    ax.axis("off")

In [ ]:
display_cv2_img(img)

In [ ]:
cap.release()

# Display multiple frames from the video

In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(30, 20))
axs = axs.flatten()

cap = cv2.VideoCapture("026c7465-309f6d33.mp4")
n_frames = int(cap.get(cv2.cv2.CAP_PROP_FRAME_COUNT))

img_idx = 0
for frame in range(n_frames):
    ret, img = cap.read()
    if ret == False:
        break
    if frame % 100 == 0:
        axs[img_idx].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axs[img_idx].set_title(f'Frame: {frame}')
        axs[img_idx].axis('off')
        img_idx += 1

plt.tight_layout()
plt.show()
cap.release()

# Add Annotations to Video Images

In [ ]:
labels = pd.read_csv('../input/driving-video-with-object-tracking/mot_labels.csv',
                     low_memory=False)
video_labels = (
    labels.query('videoName == "026c7465-309f6d33"').reset_index(drop=True).copy()
)
video_labels["video_frame"] = (video_labels["frameIndex"] * 11.9).round().astype("int")

In [ ]:
video_labels["category"].value_counts()

In [ ]:
# Pull frame 1035

cap = cv2.VideoCapture("026c7465-309f6d33.mp4")
n_frames = int(cap.get(cv2.cv2.CAP_PROP_FRAME_COUNT))

img_idx = 0
for frame in range(n_frames):
    ret, img = cap.read()
    if ret == False:
        break
    if frame == 1035:
        break
cap.release()

In [ ]:
display_cv2_img(img)

In [ ]:
img_example = img.copy()
frame_labels = video_labels.query('video_frame == 1035')
for i, d in frame_labels.iterrows():
    pt1 = int(d['box2d.x1']), int(d['box2d.y1'])
    pt2 = int(d['box2d.x2']), int(d['box2d.y2'])
    cv2.rectangle(img_example, pt1, pt2, (0, 0, 255), 3)

display_cv2_img(img_example)

# Displaying Colored by Category

In [ ]:
color_map = {
    "car": (0, 0, 255),
    "truck": (0, 0, 100),
    "pedestrian": (255, 0, 0),
    "other vehicle": (0, 0, 150),
    "rider": (200, 0, 0),
    "bicycle": (0, 255, 0),
    "other person": (200, 0, 0),
    "trailer": (0, 150, 150),
    "motorcycle": (0, 150, 0),
    "bus": (0, 0, 100),
}

img_example = img.copy()
frame_labels = video_labels.query('video_frame == 1035')
for i, d in frame_labels.iterrows():
    pt1 = int(d['box2d.x1']), int(d['box2d.y1'])
    pt2 = int(d['box2d.x2']), int(d['box2d.y2'])
    color = color_map[d['category']]
    cv2.rectangle(img_example, pt1, pt2, color, 3)

display_cv2_img(img_example)

## Adding Text

In [ ]:
frame_labels = video_labels.query("video_frame == @frame")
font = cv2.FONT_HERSHEY_TRIPLEX
img_example = img.copy()
for i, d in frame_labels.iterrows():
    pt1 = int(d["box2d.x1"]), int(d["box2d.y1"])
    pt2 = int(d["box2d.x2"]), int(d["box2d.y2"])
    color = color_map[d["category"]]
    img_example = cv2.rectangle(img_example, pt1, pt2, color, 3)
    pt_text = int(d["box2d.x1"]) + 5, int(d["box2d.y1"] + 10)
    img_example = cv2.putText(img_example, d["category"], pt_text, font, 0.5, color)
display_cv2_img(img_example)
cap.release()

# Label and output Annotated Video

In [ ]:
def add_annotations(img, frame, video_labels):
    max_frame = video_labels.query("video_frame <= @frame")["video_frame"].max()
    frame_labels = video_labels.query("video_frame == @max_frame")
    for i, d in frame_labels.iterrows():
        pt1 = int(d["box2d.x1"]), int(d["box2d.y1"])
        pt2 = int(d["box2d.x2"]), int(d["box2d.y2"])
        color = color_map[d["category"]]
        img = cv2.rectangle(img, pt1, pt2, color, 3)
    return img

In [ ]:
!rm -r out_test.mp4

In [ ]:
VIDEO_CODEC = "mp4v"
fps = 59.94
width = 1280
height = 720
out = cv2.VideoWriter("out_test.mp4",
                cv2.VideoWriter_fourcc(*VIDEO_CODEC),
                fps,
                (width, height))

cap = cv2.VideoCapture("026c7465-309f6d33.mp4")
n_frames = int(cap.get(cv2.cv2.CAP_PROP_FRAME_COUNT))

for frame in tqdm(range(n_frames), total=n_frames):
    ret, img = cap.read()
    if ret == False:
        break
    img = add_annotations(img, frame, video_labels)
    out.write(img)
out.release()
cap.release()

In [ ]:
!ls -GFlash -color

# Convert our labeled output to mp4 and view

In [ ]:
tmp_output_path = "out_test.mp4"
output_path = "out_test_compressed.mp4"
subprocess.run(
    [
        "ffmpeg",
        "-i",
        tmp_output_path,
        "-crf",
        "18",
        "-preset",
        "veryfast",
        "-vcodec",
        "libx264",
        output_path,
        '-loglevel',
        'quiet'
    ]
)

In [ ]:
!ls -GFlash --color

In [ ]:
ipd.Video('out_test_compressed.mp4', width=600)

# The End